# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-02-26 17:32:34] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='0.0.0.0', port=32054, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=475652590, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, en

[2025-02-26 17:32:53 TP0] Init torch distributed begin.
[2025-02-26 17:32:54 TP0] Load weight begin. avail mem=40.73 GB


[2025-02-26 17:32:54 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-02-26 17:32:54 TP0] Using model weights format ['*.safetensors']
[2025-02-26 17:32:54 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.02it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.02it/s]

[2025-02-26 17:32:55 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=38.19 GB
[2025-02-26 17:32:55 TP0] KV Cache is allocated. K size: 0.31 GB, V size: 0.31 GB.
[2025-02-26 17:32:55 TP0] Memory pool end. avail mem=37.40 GB


[2025-02-26 17:32:56 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072
[2025-02-26 17:32:56] INFO:     Started server process [1891647]
[2025-02-26 17:32:56] INFO:     Waiting for application startup.
[2025-02-26 17:32:56] INFO:     Application startup complete.
[2025-02-26 17:32:56] INFO:     Uvicorn running on http://0.0.0.0:32054 (Press CTRL+C to quit)
[2025-02-26 17:32:56] INFO:     127.0.0.1:57172 - "GET /v1/models HTTP/1.1" 200 OK


[2025-02-26 17:32:57] INFO:     127.0.0.1:57182 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-02-26 17:32:57 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-26 17:32:58] INFO:     127.0.0.1:57186 - "POST /generate HTTP/1.1" 200 OK
[2025-02-26 17:32:58] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](../references/sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-02-26 17:33:01 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 1, cache hit rate: 6.67%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-26 17:33:01 TP0] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 7.35, #queue-req: 0
[2025-02-26 17:33:01] INFO:     127.0.0.1:41866 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-02-26 17:33:01] INFO:     127.0.0.1:41878 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-02-26 17:33:01] INFO:     127.0.0.1:41884 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-02-26 17:33:01 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, cache hit rate: 6.25%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-26 17:33:01] INFO:     127.0.0.1:41896 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-02-26 17:33:01] INFO:     127.0.0.1:41910 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-02-26 17:33:01] INFO:     127.0.0.1:41912 - "POST /flush_cache HTTP/1.1" 200 OK
[2025-02-26 17:33:01 TP0] Cache flushed successfully!


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "meta-llama/Llama-3.2-1B"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."
assert response.json().keys() == {"success", "message"}

[2025-02-26 17:33:01] Start update_weights. Load format=auto
[2025-02-26 17:33:01 TP0] Update engine weights online from disk begin. avail mem=55.67 GB


[2025-02-26 17:33:01 TP0] Using model weights format ['*.safetensors']


[2025-02-26 17:33:02 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.95it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.95it/s]

[2025-02-26 17:33:02 TP0] Update weights end.
[2025-02-26 17:33:02 TP0] Cache flushed successfully!
[2025-02-26 17:33:02] INFO:     127.0.0.1:41916 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "meta-llama/Llama-3.2-1B-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "meta-llama/Llama-3.2-1B-wrong"
    " (repository not found)."
)

[2025-02-26 17:33:02] Start update_weights. Load format=auto
[2025-02-26 17:33:02 TP0] Update engine weights online from disk begin. avail mem=55.67 GB


[2025-02-26 17:33:02 TP0] Failed to get weights iterator: meta-llama/Llama-3.2-1B-wrong (repository not found).
[2025-02-26 17:33:02] INFO:     127.0.0.1:41920 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [10]:
terminate_process(server_process)

embedding_process, port = launch_server_cmd(
    """
python -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-7B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-02-26 17:33:15] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-7B-instruct', chat_template=None, is_embedding=True, revision=None, skip_tokenizer_init=False, host='0.0.0.0', port=37849, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=260298418, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, 

[2025-02-26 17:33:20] Downcasting torch.float32 to torch.float16.


[2025-02-26 17:33:32 TP0] Downcasting torch.float32 to torch.float16.


[2025-02-26 17:33:33 TP0] Overlap scheduler is disabled for embedding models.
[2025-02-26 17:33:33 TP0] Downcasting torch.float32 to torch.float16.
[2025-02-26 17:33:33 TP0] Init torch distributed begin.


[2025-02-26 17:33:33 TP0] Load weight begin. avail mem=58.65 GB


...

Loading safetensors checkpoint shards:  86% Completed | 6/7 [00:10<00:01,  1.60s/it]


Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:11<00:00,  1.55s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:11<00:00,  1.70s/it]

[2025-02-26 17:33:46 TP0] Load weight end. type=Qwen2ForCausalLM, dtype=torch.float16, avail mem=64.38 GB
[2025-02-26 17:33:46 TP0] KV Cache is allocated. K size: 0.55 GB, V size: 0.55 GB.
[2025-02-26 17:33:46 TP0] Memory pool end. avail mem=63.07 GB


[2025-02-26 17:33:46 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072
[2025-02-26 17:33:47] INFO:     Started server process [1894129]
[2025-02-26 17:33:47] INFO:     Waiting for application startup.
[2025-02-26 17:33:47] INFO:     Application startup complete.
[2025-02-26 17:33:47] INFO:     Uvicorn running on http://0.0.0.0:37849 (Press CTRL+C to quit)


[2025-02-26 17:33:47] INFO:     127.0.0.1:55718 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-02-26 17:33:47 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-26 17:33:47] INFO:     127.0.0.1:55744 - "GET /v1/models HTTP/1.1" 200 OK


[2025-02-26 17:33:48] INFO:     127.0.0.1:55732 - "POST /encode HTTP/1.1" 200 OK
[2025-02-26 17:33:48] The server is fired up and ready to roll!


In [11]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-02-26 17:33:53 TP0] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-26 17:33:53] INFO:     127.0.0.1:56050 - "POST /encode HTTP/1.1" 200 OK


In [12]:
terminate_process(embedding_process)

## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [13]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-02-26 17:34:06] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, is_embedding=True, revision=None, skip_tokenizer_init=False, host='0.0.0.0', port=30954, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=295706067, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, s

[2025-02-26 17:34:24 TP0] Overlap scheduler is disabled for embedding models.
[2025-02-26 17:34:24 TP0] Init torch distributed begin.


[2025-02-26 17:34:24 TP0] Load weight begin. avail mem=78.81 GB


[2025-02-26 17:34:24 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-02-26 17:34:25 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.58it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.32it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.42it/s]

[2025-02-26 17:34:28 TP0] Load weight end. type=LlamaForSequenceClassification, dtype=torch.bfloat16, avail mem=64.68 GB
[2025-02-26 17:34:28 TP0] KV Cache is allocated. K size: 1.25 GB, V size: 1.25 GB.
[2025-02-26 17:34:28 TP0] Memory pool end. avail mem=61.96 GB


[2025-02-26 17:34:29 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072
[2025-02-26 17:34:29] INFO:     Started server process [1895707]
[2025-02-26 17:34:29] INFO:     Waiting for application startup.
[2025-02-26 17:34:29] INFO:     Application startup complete.
[2025-02-26 17:34:29] INFO:     Uvicorn running on http://0.0.0.0:30954 (Press CTRL+C to quit)


[2025-02-26 17:34:30] INFO:     127.0.0.1:37040 - "GET /v1/models HTTP/1.1" 200 OK
[2025-02-26 17:34:30] INFO:     127.0.0.1:37042 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-02-26 17:34:30 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-26 17:34:31] INFO:     127.0.0.1:37046 - "POST /encode HTTP/1.1" 200 OK
[2025-02-26 17:34:31] The server is fired up and ready to roll!


In [14]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-02-26 17:34:40 TP0] Prefill batch. #new-seq: 1, #new-token: 68, #cached-token: 1, cache hit rate: 1.32%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-26 17:34:40 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 62, cache hit rate: 43.45%, token usage: 0.00, #running-req: 1, #queue-req: 0
[2025-02-26 17:34:40] INFO:     127.0.0.1:39420 - "POST /classify HTTP/1.1" 200 OK


In [15]:
terminate_process(reward_process)

## Skip Tokenizer and Detokenizer

SGLang Runtime also supports skip tokenizer and detokenizer. This is useful in cases like integrating with RLHF workflow.

In [16]:
tokenizer_free_server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --skip-tokenizer-init
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-02-26 17:34:53] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=True, host='127.0.0.1', port=39804, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=922992250, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, e

[2025-02-26 17:35:11 TP0] Init torch distributed begin.
[2025-02-26 17:35:11 TP0] Load weight begin. avail mem=78.81 GB


[2025-02-26 17:35:12 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-02-26 17:35:12 TP0] Using model weights format ['*.safetensors']
[2025-02-26 17:35:12 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.04it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.04it/s]

[2025-02-26 17:35:13 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=76.37 GB
[2025-02-26 17:35:13 TP0] KV Cache is allocated. K size: 0.31 GB, V size: 0.31 GB.
[2025-02-26 17:35:13 TP0] Memory pool end. avail mem=75.58 GB


[2025-02-26 17:35:13 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072
[2025-02-26 17:35:13] INFO:     Started server process [1896653]
[2025-02-26 17:35:13] INFO:     Waiting for application startup.
[2025-02-26 17:35:13] INFO:     Application startup complete.
[2025-02-26 17:35:13] INFO:     Uvicorn running on http://127.0.0.1:39804 (Press CTRL+C to quit)


[2025-02-26 17:35:13] INFO:     127.0.0.1:58792 - "GET /v1/models HTTP/1.1" 200 OK


[2025-02-26 17:35:14] INFO:     127.0.0.1:58798 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-02-26 17:35:14 TP0] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-26 17:35:17] INFO:     127.0.0.1:58804 - "POST /generate HTTP/1.1" 200 OK
[2025-02-26 17:35:17] The server is fired up and ready to roll!


In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

input_text = "What is the capital of France?"

input_tokens = tokenizer.encode(input_text)
print_highlight(f"Input Text: {input_text}")
print_highlight(f"Tokenized Input: {input_tokens}")

response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "input_ids": input_tokens,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 256,
            "stop_token_ids": [tokenizer.eos_token_id],
        },
        "stream": False,
    },
)
output = response.json()
output_tokens = output["token_ids"]

output_text = tokenizer.decode(output_tokens, skip_special_tokens=False)
print_highlight(f"Tokenized Output: {output_tokens}")
print_highlight(f"Decoded Output: {output_text}")
print_highlight(f"Output Text: {output['meta_info']['finish_reason']}")

[2025-02-26 17:35:19 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-26 17:35:19 TP0] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 6.55, #queue-req: 0


[2025-02-26 17:35:19 TP0] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 167.55, #queue-req: 0
[2025-02-26 17:35:20] INFO:     127.0.0.1:58812 - "POST /generate HTTP/1.1" 200 OK


In [18]:
terminate_process(tokenizer_free_server_process)